In [1]:
import numpy as np
import LCDMSphere

r0 = 2.818e-13 # classical electron radius in cm
m_e = 9.10938e-28 # electron mass in gram
c = 2.998e10 # speed of light in cm/s
h = 6.6261e-27 # planck constant in cm^2 g s-1
k = 1.3808e-16 # boltzmann constant in cm^2 g s^-2 K^-1
T_CMB = 2.725 # CMB temperature today in K

H_0 = 2.184e-18 # current hubble constant in s^-1
H_r = 3.24076e-18 # 100 km/s/Mpc to 1/s
h_0 = H_0 / H_r
omega_b_0 = 0.0224 / h_0 / h_0  # current baryon abundance
m_p = 1.67262193269e-24 # proton mass in g
G = 6.6743e-8 # gravitational constant in cm^3 g^-1 s^-2
f_He = 0.079 # helium mass fraction
f_H = 0.76 # hydrogen fraction

z_reion_He = 3 # redshift that He is fully ionized
z_reion_H = 11 # redshift that H is fully ionized

aR = 8 * np.pi**5 * k**4 / (15 * c**3 * h**3) # radiation constant

In [2]:
def E_loss_inv_Compton(E, z):
    T_CMB_z = T_CMB * (1 + z)
    return -32 * np.pi * r0 * r0 * aR * T_CMB_z**4 / (9 * m_e * c) * (2 + E / m_e / c / c) * E

In [13]:
def E_loss_coll_free(E, z, overdensity):
    E_loss_free = 0 # initialize
    E_loss_coll = 0 # initialize
    
    omega_b = omega_b_0 * (1 + z)**3
    mean_n_b = 3 * omega_b * H_0 * H_0 / (8 * np.pi * G * m_p) # mean number density of baryons at the redshift
    n_b = mean_n_b * overdensity # local number density of baryons at the redshitf
    
    print("n_b = "+str(n_b)) # for test
    
    n_H = n_b * (1 - f_He) # nuclei in hydrogen
    n_He = n_b * f_He / 4 # nuclei in helium, assume He is all 4He
    
    print("n_He = "+str(n_He)) # for test
    
    N_ex = [1, 1, 2] # number of bound electrons, H0, He+, He0 accordingly
    I_x_eV = [15.0, 59.9, 42.3] # geometric mean excitation energy for H0, He+, He0 accordingly
    I_x = [x * 1.602e-12 for x in I_x_eV] # 1eV = 1.602e-12 erg
    
    if z <= z_reion_He: # fully ionized
        n_e = n_H + 2 * n_He
        n_x = [0, 0, 0] # number density of H0, He+, He0 accordingly
    if z_reion_He < z <= z_reion_H: # He is singly ionized
        n_e = n_H + n_He
        n_x = [0, n_He, 0]
    if z > z_reion_H: # neutral
        n_e = 0
        n_x = [n_H, 0, n_He]
    
    gamma = E / (m_e * c * c) + 1
    beta = np.sqrt(E * (E + 2 * m_e * c * c)) / (E + m_e * c * c)
    omega_p = c * np.sqrt(4 * np.pi * r0 * n_e) # plasma frequency
    I = h * omega_p # mean excitation energy
    tau = E / (m_e * c * c)
    F = (1 - beta * beta) * (1 + tau * tau / 8 - (2 * tau + 1) * np.log(2)) # Moller scattering formula
    zeta = 2 * np.log(gamma) - beta * beta # density correction
    
    if z <= z_reion_H:
        E_loss_free = -2 * np.pi * n_e * r0 * r0 * m_e * c**3 / beta * (np.log(E * E / I / I) + np.log(1 + tau / 2) + F - zeta)
    if z > z_reion_He:
        for i in range(3):
            E_loss_coll += -2 * np.pi * r0 * r0 * m_e * c**3 / beta * (N_ex[i] * n_x[i] * (np.log(E * E / I_x[i] / I_x[i]) + np.log(1 + tau / 2) + F))
            print('E_loss_coll ='+str(E_loss_coll)) # for test
    
    return E_loss_free + E_loss_coll

In [14]:
def E_loss(E, z, overdensity):
    E1 = E_loss_inv_Compton(E, z) # energy loss due to inverse Compton
    E2 = E_loss_coll_free(E, z, overdensity) # energy loss due to collision
    return E1 + E2

In [15]:
E_loss(1e-6, 1, 1)

n_b = 2.012378316030844e-06
n_He = 3.974447174160917e-08


-2.915938124473395e-24

In [16]:
E_loss(1e-6, 5, 1)

n_b = 5.433421453283278e-05
n_He = 1.0731007370234475e-06
E_loss_coll =0.0
E_loss_coll =-2.757752033239887e-25
E_loss_coll =-2.757752033239887e-25


-1.132051049857061e-22

In [17]:
E_loss(1e-6, 12, 1)

n_b = 0.0005526493950399705
n_He = 1.0914825552039416e-05
E_loss_coll =-1.501376919731757e-22
E_loss_coll =-1.501376919731757e-22
E_loss_coll =-1.559559991254549e-22


-1.4030567328827561e-21

In [18]:
# for E = 1e-6 erg approx  0.6 MeV, stopping power due of Helium due to collision is 1.884 MeV approx 3.02e-6 erg
def test_sec(E, n_b, stop_power):
    m_He = 6.6464731e-24 # Helium mass in gram
    v = np.sqrt(1 - 1 / (E / m_e / c / c + 1)**2) * c * c # E = m_e*c*c*(1/np.sqrt(1-v*v/c/c)-1), in cm/s
    rho_He = n_b * f_He * m_He # Helium mass density
    return -rho_He * v * stop_power # energy loss due to He0

E_z5 = test_sec(1e-6, 5.433421453283278e-05, 3.02e-6)
E_z12 = test_sec(1e-6, 0.0005526493950399705, 3.02e-6)
print(E_z5)
print(E_z12)

-6.91488651538376e-14
-7.033335960323204e-13
